In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import List, Dict, Optional

In [3]:
# 1. 만든 툴킷 가져오기
from dolmary_schedule_toolkit import DolmaryScheduleToolkit

toolkits = DolmaryScheduleToolkit()
tools = toolkits.get_tools()
tools

[AddToDoTool(), ViewToDoTool(), DeleteTodoTool(), CompleteTodoTool()]

In [4]:
# 2. 모델 설정
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 돌마리 스케줄을 관리하는 AI 비서야. 주어지는 도구를 잘 활용해서 사용자의 요청에 잘 응답하도록 해"),
        MessagesPlaceholder(variable_name = "history"),     # ("placeholder", "{history}"),
        ("human", "{question}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

# 4. agent 생성
agent = create_openai_tools_agent(
    model,
    tools,
    prompt
)

executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

In [5]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 대화내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}

def get_store(session_id : str):
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

agent_history = RunnableWithMessageHistory(
    executor,
    lambda session_id: get_store(session_id),
    input_messages_key = "question",
    history_messages_key = "history"
)

In [6]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘"
}, config = cfg)



> Entering new AgentExecutor chain...

Invoking: `add_toDo` with `{'item': '12시 30분에 파워댄스 운동회 일정'}`


12시 30분에 파워댄스 운동회 일정이 돌마리 할일 스케줄에 등록되었습니다오늘 12시 30분에 파워댄스 운동회 일정이 돌마리 스케줄에 등록되었습니다. 다른 일정도 도와드릴까요?

> Finished chain.


{'question': '오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘',
 'history': [],
 'output': '오늘 12시 30분에 파워댄스 운동회 일정이 돌마리 스케줄에 등록되었습니다. 다른 일정도 도와드릴까요?'}

In [7]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "돌마리 스케줄 전체 보여줘"
}, config = cfg)



> Entering new AgentExecutor chain...

Invoking: `view_toDos` with `{}`


할일 목록은 : 12시 30분에 파워댄스 운동회 일정
입니다현재 돌마리 스케줄에는 "12시 30분에 파워댄스 운동회 일정"이 등록되어 있습니다. 다른 일정 관리가 필요하신가요?

> Finished chain.


{'question': '돌마리 스케줄 전체 보여줘',
 'history': [HumanMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘 12시 30분에 파워댄스 운동회 일정이 돌마리 스케줄에 등록되었습니다. 다른 일정도 도와드릴까요?', additional_kwargs={}, response_metadata={})],
 'output': '현재 돌마리 스케줄에는 "12시 30분에 파워댄스 운동회 일정"이 등록되어 있습니다. 다른 일정 관리가 필요하신가요?'}

In [8]:
from dolmary_schedule_toolkit.toolkit import DolmaryScheduleToolkit

toolkits2 = DolmaryScheduleToolkit()
tools2 = toolkits2.get_tools()
tools2

[AddToDoTool(), ViewToDoTool(), DeleteTodoTool(), CompleteTodoTool()]

In [7]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "오늘 돌마리 운동회 파워댄스 일정 삭제해줘"
}, config = cfg)



> Entering new AgentExecutor chain...

Invoking: `delete_toDo` with `{'item': '운동회 12시 30분 파워댄스'}`


운동회 12시 30분 파워댄스이 돌마리 스케줄에서 삭제되었습니다오늘 돌마리 운동회 파워댄스 일정이 삭제되었습니다. 다른 일정 관리가 필요하시면 말씀해 주세요.

> Finished chain.


{'question': '오늘 돌마리 운동회 파워댄스 일정 삭제해줘',
 'history': [HumanMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정이 등록되었습니다. 더 도와드릴 일이 있을까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='돌마리 스케줄 전체 보여줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='현재 돌마리 스케줄에는 "운동회 12시 30분 파워댄스" 일정이 등록되어 있습니다. 다른 일정도 확인하거나 추가할까요?', additional_kwargs={}, response_metadata={})],
 'output': '오늘 돌마리 운동회 파워댄스 일정이 삭제되었습니다. 다른 일정 관리가 필요하시면 말씀해 주세요.'}

In [9]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "오늘 돌마리 운동회 파워댄스 일정 완료상태로 표시해줘"
}, config = cfg)



> Entering new AgentExecutor chain...

Invoking: `complete_toDo` with `{'item': '12시 30분에 파워댄스 운동회 일정'}`


'12시 30분에 파워댄스 운동회 일정' 항목을 완료 처리했습니다.오늘 돌마리 운동회 파워댄스 일정을 완료 상태로 표시했습니다. 다른 일정도 도와드릴까요?

> Finished chain.


{'question': '오늘 돌마리 운동회 파워댄스 일정 완료상태로 표시해줘',
 'history': [HumanMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘 12시 30분에 파워댄스 운동회 일정이 돌마리 스케줄에 등록되었습니다. 다른 일정도 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='돌마리 스케줄 전체 보여줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='현재 돌마리 스케줄에는 "12시 30분에 파워댄스 운동회 일정"이 등록되어 있습니다. 다른 일정 관리가 필요하신가요?', additional_kwargs={}, response_metadata={})],
 'output': '오늘 돌마리 운동회 파워댄스 일정을 완료 상태로 표시했습니다. 다른 일정도 도와드릴까요?'}

In [10]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "돌마리 스케줄 전체 보여줘"
}, config = cfg)



> Entering new AgentExecutor chain...

Invoking: `view_toDos` with `{}`


할일 목록은 : 12시 30분에 파워댄스 운동회 일정 [완료]
입니다현재 돌마리 스케줄에는 "12시 30분에 파워댄스 운동회 일정"이 완료 상태로 등록되어 있습니다. 다른 일정도 확인하거나 관리할까요?

> Finished chain.


{'question': '돌마리 스케줄 전체 보여줘',
 'history': [HumanMessage(content='오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘 12시 30분에 파워댄스 운동회 일정이 돌마리 스케줄에 등록되었습니다. 다른 일정도 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='돌마리 스케줄 전체 보여줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='현재 돌마리 스케줄에는 "12시 30분에 파워댄스 운동회 일정"이 등록되어 있습니다. 다른 일정 관리가 필요하신가요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='오늘 돌마리 운동회 파워댄스 일정 완료상태로 표시해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘 돌마리 운동회 파워댄스 일정을 완료 상태로 표시했습니다. 다른 일정도 도와드릴까요?', additional_kwargs={}, response_metadata={})],
 'output': '현재 돌마리 스케줄에는 "12시 30분에 파워댄스 운동회 일정"이 완료 상태로 등록되어 있습니다. 다른 일정도 확인하거나 관리할까요?'}